In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# %matplotlib widget


In [2]:
# Standard Library Imports
import os
import csv
from datetime import datetime, timedelta, timezone
from collections import defaultdict
from collections import OrderedDict
from itertools import product
from itertools import groupby
import itertools
import pickle
import glob
import chardet
import copy

# Third-Party Library Imports
import numpy as np
import matplotlib.pyplot as plt
# Set a professional style for the plot
plt.style.use('_mpl-gallery')
import matplotlib.dates as mdaates
import matplotlib.gridspec as gridspec
import matplotlib.cm as cm
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
from obspy import read, UTCDateTime
from obspy.signal.trigger import classic_sta_lta, trigger_onset, plot_trigger
from obspy import Trace
from obspy.imaging.spectrogram import spectrogram
import pandas as pd
from geopy.distance import geodesic
from tqdm.auto import tqdm
from icecream import ic
import seaborn as sns
from scipy.stats import norm, kstest, skew, kurtosis
import warnings
warnings.filterwarnings("ignore")

# Local Imports
from energy.utils_general import *
from energy.utils_energy import *
from energy.preprocessing import *
from energy.metrics import *

A tener en consideración. Las rutas a los archivos están guardadas de la siguiente forma: *waveform_M/event_ID/channel/station/archivos.mseed*


# 1. Creamos un diccionario con las coordenadas de las estaciones del litoral a partir del archivo Excel.

In [3]:
v_P = 8.064

file_stations_coord = "Estaciones_Litoral.xlsx"
df = pd.read_excel(file_stations_coord)
stations_coord_all = df.set_index('Estacion')[['Lat', 'Lon']].apply(tuple, axis=1).to_dict()
stations_names = list(stations_coord_all.keys())

# 2. Trabajar con la base de datos

### 2.1. La función z_channel_dict entrega un nested dictionary donde la primera capa de llaves indica la magnitud, la segunda capa el evento, la tercera la red de la estación que la captó, y finalmente el valor es la traza ya leída por obspy

In [4]:
# Get the path to all the folders from the folder BD paper that is in the current directory
folders = glob.glob('BD paper/*')
# Get all the folders in the BD paper folder that start with waveform 
folders_signals = [folder for folder in folders if folder.startswith('BD paper/waveform')]
folders_signals = sorted(folders_signals)

In [5]:
folders_signals

['BD paper/waveform_4p1_0.5',
 'BD paper/waveform_4p2_0.5',
 'BD paper/waveform_5_0.5',
 'BD paper/waveform_6-9_0.5']

Esta función se puede/debe optimizar. Usando os.walk probablemente sale más rápido. No sé si existe alguna forma de hacerla más generalizable eso si

In [50]:
def z_channel_dict(folders_signals):


    result_dict = {}

    for folder in folders_signals:
        waveform_type = os.path.split(folder)[-1]
        events_id = glob.glob(os.path.join(folder, '*'))
        #ic(waveform_type, events_id)

        for event_id in events_id:
            events_name = os.path.split(event_id)[-1]
            network_path = glob.glob(os.path.join(event_id, '*'))

            for network_folder in network_path:
                network_name = os.path.split(network_folder)[-1]
                
                stations_path = glob.glob(os.path.join(network_folder, '*'))
                #ic(stations_path)

                for station_path in stations_path:
                    stations_names = os.path.split(station_path)[-1]
                    stations_path = glob.glob(os.path.join(station_path, '*'))

                    stations_ch_z = [station_z for station_z in stations_path if 'BHZ' in station_z]

                    # Read and store traces using ObsPy
                    traces = []
                    for station_z_path in stations_ch_z:
                        trace = read(station_z_path)[0]  # Assuming one trace per file
                        traces.append(trace)


                    result_dict.setdefault(waveform_type, {}).setdefault(events_name, {}).setdefault(network_name, {}).setdefault(stations_names, traces)

    return result_dict
result_dict = z_channel_dict(folders_signals)

In [84]:
waveform_type = list(result_dict.keys())

# get all the events ids consideting waveform type is a list. And make events_ids a dictionary with the waveform type as keys
events_ids = {waveform_type: [list(result_dict[waveform_type].keys())]for waveform_type in waveform_type}


### 2.2. La siguiente función pone a todas las traces de TODOS los eventos en una fila, esto lo tengo a priori, quizás habrá que arreglarlo

In [62]:
def get_all_traces(nested_dict):
    traces = []
    for value in nested_dict.values():
        if isinstance(value, dict):
            traces.extend(get_all_traces(value))
        else:
            traces.extend(value)
    return traces

# Get all ObsPy Trace objects in the dictionary
all_traces = get_all_traces(result_dict)

In [65]:
folders

['BD paper/catalogos',
 'BD paper/waveform_6-9_0.5',
 'BD paper/waveform_4p1_0.5',
 'BD paper/waveform_4p2_0.5',
 'BD paper/waveform_5_0.5']

### 2.3 El siguiente código es para crear un nuevo archivo CSV que contenga solo los siguientes datos:
- EventId
- Time
- Latitud
- Longitud
- Magnitud
- Estación (la que aparece automáticamente en el catálogo)
- Estación más cercana (calculada con geodisic)

In [66]:
def merge_coordinates(files_to_add, files_with_coord):
    
    for file_to_add, file_with_coords in zip(files_to_add, files_with_coord):
        df_to_add = pd.read_csv(file_to_add, sep=',')
        df_with_coords = pd.read_csv(file_with_coords, sep='|')

        df_to_add['Time'] = pd.to_datetime(df_to_add['Time'])
        df_with_coords['Time'] = pd.to_datetime(df_with_coords['Time'])

        df_to_add = df_to_add.set_index('Time')

        # Merge df_to_add with df_with_coords on 'Time', keeping only 'Latitude' and 'Longitude' from df_with_coords
        df_to_add = pd.merge(df_to_add, df_with_coords[['Time','Latitude', 'Longitude']], on='Time', how='left', suffixes=('', '_y'))

        # Drop the '_y' columnsbb
        df_to_add = df_to_add.drop(columns=['Latitude_y', 'Longitude_y'], errors='ignore')

        df_to_add.to_csv(file_to_add, index=False, sep = ',')


# Get all the csv files in the BD paper folder that have the word "cercanos" in their name
files_raw = sorted(glob.glob(os.path.join('BD paper', 'catalogos','*cercanos*.csv')))
# Get all the csv files in the BD paper folder that have the word "full_data" in their name
files_full_data = sorted(glob.glob(os.path.join('BD paper', 'lista de datos','*full_data*.csv')))


# Call the function to merge coordinates for each pair of files
merge_coordinates(files_raw, files_full_data)


In [86]:
# Get all the csv files in the BD paper folder that have the word "cercanos" in their name
files_raw = sorted(glob.glob(os.path.join('BD paper', 'catalogos','*.csv')))

files_raw

['BD paper/catalogos/Eventos_descargados_4p1.csv',
 'BD paper/catalogos/Eventos_descargados_4p2.csv',
 'BD paper/catalogos/Eventos_descargados_5.csv']

In [ ]:


# Lee el archivo Excel original
df_original = pd.read_excel("tu_archivo_original.xlsx")

# Diccionario de eventos
events_ids = {
    'waveform_4p1_0.5': [['5190277', '10071115', '5178072', '11019653', '11090343', '11340398', '5185702', '10896756']],
    'waveform_4p2_0.5': [['4597464', '5174164', '11137931', '11207110', '10000295', '11272932', '11204853', '4600957', '5188856', '11304668', '11252658']]
}

# Filtrar y seleccionar columnas
filtered_df = pd.DataFrame()
for waveform_type, event_ids_list in events_ids.items():
    for event_ids in event_ids_list:
        filtered_df = filtered_df.append(df_original[df_original['#EventID'].astype(str).isin(event_ids)])

# Seleccionar solo las columnas requeridas
filtered_df = filtered_df[['#EventID', 'Time', 'Latitude', 'Longitude', 'Magnitude', 'Estacion']]

# Guardar el nuevo DataFrame en un nuevo archivo Excel
filtered_df.to_excel("nuevo_archivo.xlsx", index=False)

In [67]:
files_raw

[]

### 2.4. Ahora que ya se tienen archivos .csv con columnas "Time", "Magnitud", "Estación", "Latitud", "Longitud", podemos verificar efectivamente cuál es la estación más cercana y agregarla como última columna

In [9]:
file_path = files_raw[0]
file_path

'BD paper/lista de datos/Eventos_500_cercanos_4p1.csv'

In [21]:

# Leer el archivo CSV
with open(file_path, 'r') as f:
    df_events = pd.read_csv(f)

# Cambiar nombre de la columna "Time" a "Fecha UTC" y cosas en inglés por español
df_events = df_events.rename(columns={'Time': 'Fecha UTC'})
# Cambiar esta columna a formato UTC
df_events['Fecha UTC'] = pd.to_datetime(df_events['Fecha UTC'])
df_events = df_events.rename(columns={'Latitude': 'Latitud'})
df_events = df_events.rename(columns={'Longitude': 'Longitud'})

# Escribir en el archivo CSV
with open(file_path, 'w') as f:
    df_events.to_csv(f, index=False, sep=',')

# tomar el valor maximo y minimo de la magnitud en el dataframe
max_magnitude = df_events['Magnitud'].max()
min_magnitude = df_events['Magnitud'].min()

df_new = calculate_detection_times(df_events , stations_coord_all, v_P, magnitude_range = (min_magnitude, max_magnitude))
_, closest_sts_names = nearest_n_stations(df_new, stations_names, 1)

# Leer el archivo CSV
with open(file_path, 'r') as f:
    df_events = pd.read_csv(f)


df_events['Estación más cercana'] = closest_sts_names[0]
df_events.to_csv(file_path, index=False, sep = ',')
# Escribir en el archivo CSV
with open(file_path, 'w') as f:
    df_events.to_csv(f, index=False, sep=',')


In [12]:

def closest_station(file_path, stations_coord_all, v_P, n_closest_stations = 5):
    '''
    Actualiza todos los archivos csv con la estación más cercana a cada evento sísmico.
    '''

    for file_path in files_raw:

        # Leer el archivo CSV
        df_events = pd.read_csv(file_path)

        # Cambiar nombre de la columna "Time" a "Fecha UTC" y cosas en inglés por español
        df_events = df_events.rename(columns={'Time': 'Fecha UTC'})
        
        # Cambiar esta columna a formato UTC
        df_events['Fecha UTC'] = pd.to_datetime(df_events['Fecha UTC'])
        df_events = df_events.rename(columns={'Latitude': 'Latitud', 'Longitude': 'Longitud'})

        # Escribir en el archivo CSV
        df_events.to_csv(file_path, index=False, sep=',')

        # Tomar el valor máximo y mínimo de la magnitud en el dataframe
        max_magnitude = df_events['Magnitud'].max()
        min_magnitude = df_events['Magnitud'].min()

        df_new = calculate_detection_times(df_events, stations_coord_all, v_P, magnitude_range=(min_magnitude, max_magnitude))
        _, closest_sts_names = nearest_n_stations(df_new, stations_names, n_closest_stations)

        # Leer el archivo CSV
        df_events = pd.read_csv(file_path)

        # Agregar columnas para cada estación cercana
        for i in range(n_closest_stations):
            col_name = f'Estación más cercana {i+1}'
            df_events[col_name] = closest_sts_names[i]

        # Escribir en el archivo CSV
        df_events.to_csv(file_path, index=False, sep=',')


closest_station(files_raw, stations_coord_all, v_P)
